In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
df = pd.read_csv(
    '202401-202406-citibike-tripdata-cleaned.csv',
    dtype={
        'start_station_id': str,
        'end_station_id': str
    },
    low_memory=False
)


In [2]:
# If your dataset has a date column, convert it to datetime and set it as index
# Replace 'date' with the actual date column name if different
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

# Display basic information about the dataset
print("First 5 rows of the dataset:")
print(df.head(), "\n")

print("Dataset Information:")
print(df.info(), "\n")

print("Statistical Summary:")
print(df.describe(), "\n")

# Check for missing values in each column
print("Missing Values by Column:")
print(df.isnull().sum(), "\n")

First 5 rows of the dataset:
                  ended_at                  start_station_name member_casual  \
0  2024-01-22 18:48:10.708  Frederick Douglass Blvd & W 145 St        member   
1  2024-01-11 19:47:36.007                     W 54 St & 6 Ave        member   
2  2024-01-30 19:32:49.857                     E 11 St & Ave B        casual   
3  2024-01-27 11:38:01.213                     W 54 St & 6 Ave        member   
4  2024-01-16 15:29:26.156               Madison Ave & E 99 St        member   

     end_lng               started_at start_station_id  \
0 -73.951878  2024-01-22 18:43:19.012          7954.12   
1 -73.954823  2024-01-11 19:19:18.721          6771.13   
2 -74.008515  2024-01-30 19:17:41.693          5659.11   
3 -73.954823  2024-01-27 11:27:01.759          6771.13   
4 -73.954823  2024-01-16 15:15:41.000          7443.01   

             end_station_name  start_lat  start_lng    end_lat end_station_id  \
0  St Nicholas Ave & W 126 St  40.823072 -73.941738  40.8114

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Helper function: remove trailing zeros and an extra dot if present
def normalize_station_id(station_id):
    s = str(station_id)
    if '.' in s:
        s = s.rstrip('0').rstrip('.')
    return s

# Apply normalization to both start and end station IDs
df['start_station_id_norm'] = df['start_station_id'].apply(normalize_station_id)
df['end_station_id_norm'] = df['end_station_id'].apply(normalize_station_id)

# Investigate raw station IDs that differ only in trailing zeros
# For start station IDs
start_id_groups = df.groupby('start_station_id_norm')['start_station_id'] \
                    .nunique() \
                    .reset_index() \
                    .rename(columns={'start_station_id': 'unique_raw_ids'})
start_id_groups = start_id_groups[start_id_groups['unique_raw_ids'] > 1]
print("Start station IDs that differ only by trailing zeros:")
print(start_id_groups)

# For end station IDs
end_id_groups = df.groupby('end_station_id_norm')['end_station_id'] \
                  .nunique() \
                  .reset_index() \
                  .rename(columns={'end_station_id': 'unique_raw_ids'})
end_id_groups = end_id_groups[end_id_groups['unique_raw_ids'] > 1]
print("\nEnd station IDs that differ only by trailing zeros:")
print(end_id_groups)

# Combine station names from start and end to see if they are the same for the normalized IDs.
# We'll reshape the data so that we have one row per normalized station id with all associated station names.
start_df = df[['start_station_id_norm', 'start_station_name']].rename(
    columns={'start_station_id_norm': 'station_id_norm', 'start_station_name': 'station_name'}
)
end_df = df[['end_station_id_norm', 'end_station_name']].rename(
    columns={'end_station_id_norm': 'station_id_norm', 'end_station_name': 'station_name'}
)

# Combine both and drop duplicates
combined = pd.concat([start_df, end_df]).drop_duplicates()

# Group by normalized station id and get the unique station names
station_mapping = combined.groupby('station_id_norm')['station_name'] \
                          .unique() \
                          .reset_index()
station_mapping['num_station_names'] = station_mapping['station_name'].apply(len)

print("\nStation name mapping for each normalized station ID:")
print(station_mapping.head(20))

# Optionally, view those normalized IDs that have more than one distinct station name
inconsistent_names = station_mapping[station_mapping['num_station_names'] > 1]
print("\nNormalized station IDs with more than one station name (potential inconsistencies):")
print(inconsistent_names)


Start station IDs that differ only by trailing zeros:
     start_station_id_norm  unique_raw_ids
34                  3113.1               2
111                 3391.1               2
116                 3423.1               2
198                 3776.1               2
208                 3834.1               2
...                    ...             ...
1968                8249.1               2
2014                8358.1               2
2033                8410.1               2
2036                8416.1               2
2119                8647.1               2

[92 rows x 2 columns]

End station IDs that differ only by trailing zeros:
     end_station_id_norm  unique_raw_ids
116               3423.1               2
198               3776.1               2
208               3834.1               2
314               4129.1               2
336               4181.1               2
401               4366.1               2
418               4416.1               2
436               4455.1  

In [4]:
# Create copies of the station name columns with whitespace removed
start_df_clean = df[['start_station_id_norm', 'start_station_name']].copy()
start_df_clean['station_name_clean'] = start_df_clean['start_station_name'].str.strip()
start_df_clean = start_df_clean[['start_station_id_norm', 'station_name_clean']].rename(
    columns={'start_station_id_norm': 'station_id_norm'}
)

end_df_clean = df[['end_station_id_norm', 'end_station_name']].copy()
end_df_clean['station_name_clean'] = end_df_clean['end_station_name'].str.strip()
end_df_clean = end_df_clean[['end_station_id_norm', 'station_name_clean']].rename(
    columns={'end_station_id_norm': 'station_id_norm'}
)

# Combine cleaned station names from both start and end
combined_clean = pd.concat([start_df_clean, end_df_clean]).drop_duplicates()

# Group by normalized station ID and collect unique cleaned station names
station_mapping_clean = combined_clean.groupby('station_id_norm')['station_name_clean'] \
                                      .unique() \
                                      .reset_index()
station_mapping_clean['num_station_names'] = station_mapping_clean['station_name_clean'].apply(len)

print("Cleaned Station Name Mapping for each normalized station ID:")
print(station_mapping_clean.head(20))

# Identify normalized station IDs that still have more than one distinct name
inconsistent_names_clean = station_mapping_clean[station_mapping_clean['num_station_names'] > 1]
print("\nNormalized station IDs with more than one station name after cleaning:")
print(inconsistent_names_clean)

Cleaned Station Name Mapping for each normalized station ID:
   station_id_norm             station_name_clean  num_station_names
0       190 Morgan                   [190 Morgan]                  1
1          2733.03              [67 St & Erik Pl]                  1
2          2782.02                [5 Ave & 67 St]                  1
3          2821.05                [7 Ave & 62 St]                  1
4          2821.06                [62 St & 7 Ave]                  1
5          2832.03        [4 Ave & Shore Road Dr]                  1
6          2861.02                [57 St & 7 Ave]                  1
7          2872.02                [63 St & 5 Ave]                  1
8          2883.03           [3 Ave & Wakeman Pl]                  1
9          2898.01  [Cortelyou Rd & Stratford Rd]                  1
10         2912.08                [6 Ave & 60 St]                  1
11         2923.01                [62 St & 4 Ave]                  1
12         2932.03      [Wakeman Pl & Ridg

In [5]:
# Remove any extra whitespace and fix the typo for start_station_name and end_station_name columns
df['start_station_name'] = df['start_station_name'].str.strip().replace("Ichan Stadium Plaza", "Icahn Stadium Plaza")
df['end_station_name'] = df['end_station_name'].str.strip().replace("Ichan Stadium Plaza", "Icahn Stadium Plaza")

# Optional: Verify the change by checking the unique station names containing "Stadium Plaza"
print("Unique start station names containing 'Stadium Plaza':")
print(df[df['start_station_name'].str.contains("Stadium Plaza", na=False)]['start_station_name'].unique())

print("\nUnique end station names containing 'Stadium Plaza':")
print(df[df['end_station_name'].str.contains("Stadium Plaza", na=False)]['end_station_name'].unique())


Unique start station names containing 'Stadium Plaza':
['Icahn Stadium Plaza']

Unique end station names containing 'Stadium Plaza':
['Icahn Stadium Plaza']


In [6]:
df

,ended_at,start_station_name,member_casual,end_lng,started_at,start_station_id,end_station_name,start_lat,start_lng,end_lat,end_station_id,ride_id,rideable_type,start_station_id_norm,end_station_id_norm
0,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,member,-73.951878,2024-01-22 18:43:19.012,7954.12,St Nicholas Ave & W 126 St,40.823072,-73.941738,40.811432,7756.10,5078F3D302000BD2,electric_bike,7954.12,7756.1
1,2024-01-11 19:47:36.007,W 54 St & 6 Ave,member,-73.954823,2024-01-11 19:19:18.721,6771.13,E 74 St & 1 Ave,40.761822,-73.977036,40.768974,6953.08,814337105D37302A,electric_bike,6771.13,6953.08
2,2024-01-30 19:32:49.857,E 11 St & Ave B,casual,-74.008515,2024-01-30 19:17:41.693,5659.11,W 10 St & Washington St,40.727592,-73.979751,40.733424,5847.06,A33A920E2B10710C,electric_bike,5659.11,5847.06
3,2024-01-27 11:38:01.213,W 54 St & 6 Ave,member,-73.954823,2024-01-27 11:27:01.759,6771.13,E 74 St & 1 Ave,40.761779,-73.977144,40.768974,6953.08,A3A5FC0DD7D34D74,electric_bike,6771.13,6953.08
4,2024-01-16 15:29:26.156,Madison Ave & E 99 St,member,-73.954823,2024-01-16 15:15:41.000,7443.01,E 74 St & 1 Ave,40.789808,-73.952214,40.768974,6953.08,6F96728ECEFBDAA4,electric_bike,7443.01,6953.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18851989,2024-06-20 20:35:01.668,W 44 St & 5 Ave,member,-73.981948,2024-06-20 20:28:52.400,6551.02,E 32 St & Park Ave,40.755003,-73.980144,40.745712,6280.12,DA8BFCDCEAA4957D,classic_bike,6551.02,6280.12
18851990,2024-06-16 15:25:20.425,6 Ave & Canal St,casual,-73.981854,2024-06-16 15:12:54.352,5500.07,E 6 St & Ave B,40.722389,-74.005717,40.724537,5584.04,AE41AD4BA8B90E02,electric_bike,5500.07,5584.04
18851991,2024-06-22 08:15:09.620,6 Ave & Canal St,casual,-74.014847,2024-06-22 07:05:04.662,5500.07,West St & Liberty St,40.722438,-74.005664,40.711444,5184.08,90F1ADD977F5C19C,classic_bike,5500.07,5184.08
18851992,2024-06-21 10:33:29.781,W 13 St & 5 Ave,member,-73.981948,2024-06-21 10:23:38.519,5947.04,E 32 St & Park Ave,40.735384,-73.994781,40.745712,6280.12,0C16F8490B79B111,electric_bike,5947.04,6280.12


In [7]:
import pandas as pd

# Helper function to safely parse datetime with and without milliseconds
def safe_parse(s):
    try:
        # Try parsing assuming milliseconds are present
        return pd.to_datetime(s, format="%Y-%m-%d %H:%M:%S.%f")
    except ValueError:
        # Fallback: parse without milliseconds
        return pd.to_datetime(s, format="%Y-%m-%d %H:%M:%S")

# Convert 'started_at' and 'ended_at' using the safe parser
df['started_at'] = df['started_at'].apply(safe_parse)
df['ended_at'] = df['ended_at'].apply(safe_parse)

# Feature engineering: extract day name and hour (1-24) for both start and end times
df['start_day'] = df['started_at'].dt.day_name()
df['start_hour'] = df['started_at'].dt.hour + 1  # converting 0-23 to 1-24

df['end_day'] = df['ended_at'].dt.day_name()
df['end_hour'] = df['ended_at'].dt.hour + 1  # converting 0-23 to 1-24

# Display the first 5 rows to verify the new columns
print(df[['started_at', 'start_day', 'start_hour', 'ended_at', 'end_day', 'end_hour']].head())


In [10]:
import pandas as pd

# Define a helper function to normalize station IDs by removing trailing zeros (and a trailing dot)
def normalize_station_id(station_id):
    s = str(station_id)
    if '.' in s:
        s = s.rstrip('0').rstrip('.')
    return s

# Create normalized station ID columns for start and end, if they don't already exist
if 'start_station_id_norm' not in df.columns:
    df['start_station_id_norm'] = df['start_station_id'].apply(normalize_station_id)
if 'end_station_id_norm' not in df.columns:
    df['end_station_id_norm'] = df['end_station_id'].apply(normalize_station_id)

# Combine unique station IDs from both the start and end normalized columns
unique_stations = pd.unique(pd.concat([df['start_station_id_norm'], df['end_station_id_norm']]))
unique_stations = sorted(unique_stations)  # sort them for consistent ordering

# Create a mapping from normalized station id to a new numeric station id (starting from 1)
station_mapping = {station: idx + 1 for idx, station in enumerate(unique_stations)}

print("Total number of unique stations:", len(station_mapping))

# Map these new ids to the original dataframe
df['start_id'] = df['start_station_id_norm'].map(station_mapping)
df['end_id'] = df['end_station_id_norm'].map(station_mapping)

# Display a few rows to verify the new columns
print(df[['start_station_id', 'start_station_id_norm', 'start_id', 
          'end_station_id', 'end_station_id_norm', 'end_id']].head())

Total number of unique stations: 2268
  start_station_id start_station_id_norm  start_id end_station_id  \
0          7954.12               7954.12      1830        7756.10   
1          6771.13               6771.13      1422        6953.08   
2          5659.11               5659.11       902        5847.06   
3          6771.13               6771.13      1422        6953.08   
4          7443.01               7443.01      1638        6953.08   

  end_station_id_norm  end_id  
0              7756.1    1747  
1             6953.08    1506  
2             5847.06     958  
3             6953.08    1506  
4             6953.08    1506  


In [18]:
import pandas as pd

# Create DataFrames for station names from start and end columns, and remove any extra whitespace
start_station_names = df[['start_station_id_norm', 'start_station_name']].copy()
start_station_names['start_station_name'] = start_station_names['start_station_name'].str.strip()
start_station_names.rename(columns={'start_station_id_norm': 'norm_id', 
                                      'start_station_name': 'station_name'}, inplace=True)

end_station_names = df[['end_station_id_norm', 'end_station_name']].copy()
end_station_names['end_station_name'] = end_station_names['end_station_name'].str.strip()
end_station_names.rename(columns={'end_station_id_norm': 'norm_id', 
                                    'end_station_name': 'station_name'}, inplace=True)

# Combine the two DataFrames and drop duplicate rows
combined_station_names = pd.concat([start_station_names, end_station_names]).drop_duplicates()

# Group by the normalized station id and pick the most frequent (or first) station name.
# (If there are differences, you can adjust the aggregation as needed.)
station_names_grouped = combined_station_names.groupby('norm_id')['station_name'] \
                                              .agg(lambda x: x.mode()[0]) \
                                              .reset_index()

# Map the normalized station id to the new numeric station id using the previously created mapping.
station_names_grouped['station_id'] = station_names_grouped['norm_id'].map(station_mapping)

# Rearrange and sort the columns
station_names_grouped = station_names_grouped[['station_id', 'station_name']]
station_names_grouped = station_names_grouped.sort_values('station_id')

# Export the station names DataFrame to a CSV file
output_station_csv = 'station_names.csv'
station_names_grouped.to_csv(output_station_csv, index=False)
print(f"Station names CSV exported successfully as '{output_station_csv}'")


Station names CSV exported successfully as 'station_names.csv'


In [15]:
# Check the unique values in rideable_type
unique_types = df['rideable_type'].unique()
print("Unique rideable_type values:", unique_types)

# Define the valid types
valid_types = {'electric_bike', 'classic_bike'}

# Verify that each row's rideable_type is one of the valid types
if not set(unique_types).issubset(valid_types):
    raise ValueError("Found rideable_type values outside of electric_bike and classic_bike.")

# Map rideable_type to bike_type: 1 for electric_bike, 0 for classic_bike
df['bike_type'] = df['rideable_type'].map({'electric_bike': 1, 'classic_bike': 0})

# Display a few rows to verify the new column
print(df[['rideable_type', 'bike_type']].head())


Unique rideable_type values: ['electric_bike' 'classic_bike']
   rideable_type  bike_type
0  electric_bike          1
1  electric_bike          1
2  electric_bike          1
3  electric_bike          1
4  electric_bike          1


In [16]:
# Check unique values in member_casual
unique_member_casual = df['member_casual'].unique()
print("Unique values in member_casual:", unique_member_casual)

# Map member_casual to VIP: 'member' -> 1, 'casual' -> 0
df['VIP'] = df['member_casual'].map({'member': 1, 'casual': 0})

# Display a few rows to verify the new column
print(df[['member_casual', 'VIP']].head())


Unique values in member_casual: ['member' 'casual']
  member_casual  VIP
0        member    1
1        member    1
2        casual    0
3        member    1
4        member    1


In [17]:
df

,ended_at,start_station_name,member_casual,end_lng,started_at,start_station_id,end_station_name,start_lat,start_lng,end_lat,end_station_id,ride_id,rideable_type,start_station_id_norm,end_station_id_norm,start_id,end_id,bike_type,VIP
0,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,member,-73.951878,2024-01-22 18:43:19.012,7954.12,St Nicholas Ave & W 126 St,40.823072,-73.941738,40.811432,7756.10,5078F3D302000BD2,electric_bike,7954.12,7756.1,1830,1747,1,1
1,2024-01-11 19:47:36.007,W 54 St & 6 Ave,member,-73.954823,2024-01-11 19:19:18.721,6771.13,E 74 St & 1 Ave,40.761822,-73.977036,40.768974,6953.08,814337105D37302A,electric_bike,6771.13,6953.08,1422,1506,1,1
2,2024-01-30 19:32:49.857,E 11 St & Ave B,casual,-74.008515,2024-01-30 19:17:41.693,5659.11,W 10 St & Washington St,40.727592,-73.979751,40.733424,5847.06,A33A920E2B10710C,electric_bike,5659.11,5847.06,902,958,1,0
3,2024-01-27 11:38:01.213,W 54 St & 6 Ave,member,-73.954823,2024-01-27 11:27:01.759,6771.13,E 74 St & 1 Ave,40.761779,-73.977144,40.768974,6953.08,A3A5FC0DD7D34D74,electric_bike,6771.13,6953.08,1422,1506,1,1
4,2024-01-16 15:29:26.156,Madison Ave & E 99 St,member,-73.954823,2024-01-16 15:15:41.000,7443.01,E 74 St & 1 Ave,40.789808,-73.952214,40.768974,6953.08,6F96728ECEFBDAA4,electric_bike,7443.01,6953.08,1638,1506,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18851989,2024-06-20 20:35:01.668,W 44 St & 5 Ave,member,-73.981948,2024-06-20 20:28:52.400,6551.02,E 32 St & Park Ave,40.755003,-73.980144,40.745712,6280.12,DA8BFCDCEAA4957D,classic_bike,6551.02,6280.12,1302,1163,0,1
18851990,2024-06-16 15:25:20.425,6 Ave & Canal St,casual,-73.981854,2024-06-16 15:12:54.352,5500.07,E 6 St & Ave B,40.722389,-74.005717,40.724537,5584.04,AE41AD4BA8B90E02,electric_bike,5500.07,5584.04,841,878,1,0
18851991,2024-06-22 08:15:09.620,6 Ave & Canal St,casual,-74.014847,2024-06-22 07:05:04.662,5500.07,West St & Liberty St,40.722438,-74.005664,40.711444,5184.08,90F1ADD977F5C19C,classic_bike,5500.07,5184.08,841,709,0,0
18851992,2024-06-21 10:33:29.781,W 13 St & 5 Ave,member,-73.981948,2024-06-21 10:23:38.519,5947.04,E 32 St & Park Ave,40.735384,-73.994781,40.745712,6280.12,0C16F8490B79B111,electric_bike,5947.04,6280.12,991,1163,1,1


In [19]:
# Export the updated DataFrame to a CSV file
output_filename = 'updated_data.csv'
df.to_csv(output_filename, index=False)
print(f"Data exported successfully to {output_filename}")


Data exported successfully to updated_data.csv


In [20]:
# Select only the relevant columns for ARIMA modeling
arima_columns = ['bike_type', 'VIP', 'start_id', 'end_id', 'started_at', 'ended_at']
df_arima = df[arima_columns].copy()

# Optionally, sort by the start time if needed
df_arima.sort_values('started_at', inplace=True)

# Export the new DataFrame to a CSV file
output_filename = 'arima_features.csv'
df_arima.to_csv(output_filename, index=False)
print(f"ARIMA features CSV exported successfully as '{output_filename}'")


ARIMA features CSV exported successfully as 'arima_features.csv'


In [ ]:
# import pandas as pd

# # Helper function to safely parse datetime with and without milliseconds
# def safe_parse(s):
#     try:
#         # Try parsing assuming milliseconds are present
#         return pd.to_datetime(s, format="%Y-%m-%d %H:%M:%S.%f")
#     except ValueError:
#         # Fallback: parse without milliseconds
#         return pd.to_datetime(s, format="%Y-%m-%d %H:%M:%S")

# # Convert 'started_at' and 'ended_at' using the safe parser
# df['started_at'] = df['started_at'].apply(safe_parse)
# df['ended_at'] = df['ended_at'].apply(safe_parse)

# # Feature engineering: extract day name and hour (1-24) for both start and end times
# df['start_day'] = df['started_at'].dt.day_name()
# df['start_hour'] = df['started_at'].dt.hour + 1  # converting 0-23 to 1-24

# df['end_day'] = df['ended_at'].dt.day_name()
# df['end_hour'] = df['ended_at'].dt.hour + 1  # converting 0-23 to 1-24

# # Display the first 5 rows to verify the new columns
# print(df[['started_at', 'start_day', 'start_hour', 'ended_at', 'end_day', 'end_hour']].head())
